In [ ]:
# Sube el archivo de auth sino existe
from google.colab import files
files.upload()
# Instalar api de kaggle
!pip install kaggle
# Mueve el archivo de auth
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json # Asigna permisos solo al propietario del archivo
!kaggle datasets download -d pralabhpoudel/world-energy-consumption # Descarga el dataset
!unzip world-energy-consumption.zip # Descomprime el dataset

In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Lee el archivo CSV y almacena los datos en un DataFrame
df = pd.read_csv('World Energy Consumption.csv')

### Preprocesado y limpieza de datos

In [19]:
df.fillna(0, inplace=True) # Reemplaza los valores NaN por 0

# Lista de columnas a mantener
columns_to_keep = ['country', 'year', 'biofuel_electricity', 'biofuel_share_elec', 'biofuel_cons_change_pct', 'biofuel_share_energy', 'biofuel_cons_change_twh', 'biofuel_consumption', 'biofuel_elec_per_capita', 'biofuel_cons_per_capita', 'hydro_electricity', 'other_renewable_electricity', 'other_renewable_exc_biofuel_electricity', 'renewables_electricity', 'solar_electricity', 'wind_electricity', 'hydro_share_elec', 'hydro_cons_change_pct', 'hydro_share_energy', 'hydro_cons_change_twh', 'hydro_consumption', 'hydro_elec_per_capita', 'hydro_energy_per_capita', 'population', 'per_capita_electricity', 'renewables_elec_per_capita', 'renewables_share_elec', 'renewables_cons_change_pct', 'renewables_share_energy', 'renewables_cons_change_twh', 'renewables_consumption', 'renewables_energy_per_capita', 'solar_share_elec', 'solar_cons_change_pct', 'solar_share_energy', 'solar_cons_change_twh', 'solar_consumption', 'solar_elec_per_capita', 'solar_energy_per_capita', 'gdp', 'wind_share_elec', 'wind_cons_change_pct', 'wind_share_energy', 'wind_cons_change_twh', 'wind_consumption', 'wind_elec_per_capita', 'wind_energy_per_capita']

# Crear un nuevo DataFrame que contenga solo las columnas deseadas
df_new = df[columns_to_keep]

# Guardar el nuevo DataFrame en un archivo CSV
df_new.to_csv('World_Energy_Consumption_cleaned.csv', index=False)

df = df_new

In [20]:
import matplotlib.pyplot as plt
# establecer umbral de energía renovable
umbral = 40

# crear variable de destino
df['alta_renovable'] = df.apply(lambda row: 1 if row['renewables_share_energy'] > umbral else 0, axis=1)

# Filtrar los países con un alto porcentaje de energía renovable
alta_renovable = df[df['alta_renovable'] == 1]

In [21]:

# Seleccionar las columnas de características y la variable objetivo
X = df.loc[:, ['biofuel_share_energy', 'hydro_share_energy', 'solar_share_energy', 'wind_share_energy', 'renewables_share_energy']]
y = df['alta_renovable']

# Normalizar los datos con MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [22]:

X = df[["biofuel_share_energy", "hydro_share_energy", "solar_share_energy", "wind_share_energy", "renewables_share_energy", "per_capita_electricity", "population"]]
y = df["alta_renovable"]

# Entrenar un árbol de decisión
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba utilizando el árbol de decisión
y_pred = dtc.predict(X_test)

# Evaluar el desempeño del árbol de decisión utilizando métricas de clasificación
print("Evaluar el desempeño del árbol")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

# Entrenar una máquina de vectores de soporte
svc = SVC(random_state=42)
svc.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba utilizando la máquina de vectores de soporte
y_pred = svc.predict(X_test)

# Evaluar el desempeño de la máquina de vectores de soporte utilizando métricas de clasificación
print("Evaluar el desempeño de la máquina de vectores")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

Evaluar el desempeño del árbol
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC Score: 1.0
Evaluar el desempeño de la máquina de vectores
Accuracy: 0.9979925437338687
Precision: 0.9032258064516129
Recall: 0.875
F1 Score: 0.8888888888888888
ROC AUC Score: 0.9370658465991316
